# 数据清洗

In [1]:
import pandas as pd
import numpy as np

## 导入数据

In [2]:
train_df = pd.read_csv('meinian_round1_train_20180408.csv')
raw_data1 = pd.read_csv('meinian_round1_data_part1_20180408.txt', sep='$')
raw_data2 = pd.read_csv('meinian_round1_data_part2_20180408.txt', sep='$')
raw_data = pd.concat([raw_data1, raw_data2])

In [3]:
print(raw_data.shape)

(8104368, 3)


训练数据的结构：**患者id、检验项目id、检验结果**

In [4]:
raw_data.head()

,vid,table_id,field_results
0,002d1e4859fafd9ded2a2e1e7c839b62,2403,72.9
1,002d1e4859fafd9ded2a2e1e7c839b62,2404,166.5
2,002d1e4859fafd9ded2a2e1e7c839b62,2405,26.30
3,002d1e4859fafd9ded2a2e1e7c839b62,2420,79
4,002d1e4859fafd9ded2a2e1e7c839b62,0702,无


训练数据的输出的结构

In [5]:
train_df.head()

,vid,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
0,002d1e4859fafd9ded2a2e1e7c839b62,165,100,2.08,1.29,3.24
1,92dd479df5e30ab6a0a1cf85ac53efc3,141,97,2.64,1.36,4.75
2,6bb59d517c4c70f8f50844d24fbd0355,120,80,1.37,1.25,2.66
3,0ebb42adae512906f7e1135da734ea63,100,70,1.27,2.21,1.73
4,ebe7811e919109c42c092abbd98b4ca6,110,80,0.8,1.87,2.21


## 合并训练数据（相同的患者id、检验id）

把相同患者id和检验id的合并，以`;`分隔

In [7]:
data_compressed = raw_data.groupby(['vid','table_id'],as_index=False).apply(lambda x:";".join(map(str, x['field_results'])))

In [12]:
data_compressed = pd.DataFrame(data_compressed,columns=['field_results'])
data_compressed.shape

(7820997, 1)

In [13]:
data_fmt_all = data_compressed.unstack(fill_value=None)
data_fmt_all.columns = data_fmt_all.columns.droplevel(level=0)

In [15]:
data_fmt_all.shape

(57298, 2795)

一共有57298个病人，2795个特征

In [17]:
null_count = data_fmt_all.isnull().sum()
len(null_count[null_count<50000])

256

缺失小于50000的特征只有256个。把缺失多余50000的特征删掉。

In [18]:
data_keep_50000 = data_fmt_all.drop(labels=null_count [null_count >=50000].index,axis=1)
data_keep_50000.to_csv("data_keep_50000.csv")

In [19]:
data_keep_50000.head()

table_id,0101,0102,0113,0114,0115,0116,0117,0118,0119,0120,...,979019,979020,979021,979022,979023,A201,A202,A301,A302,A601
vid,,,,,,,,,,,,,,,,,,,,,
000330ad1f424114719b7525f400660b,双侧甲状腺大小形态正常，包膜光整，实质回声均匀，光点稍粗，未见明显异常回声。CDFI：血流显...,甲状腺彩超（含颈部淋巴细胞）未发现明显异常;前列腺:前列腺稍大膀胱、双侧输尿管未发现明显异常...,肝脏大小、形态正常，包膜光整，肝内血管走行较清晰，回声均匀。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,胰腺大小、形态正常，边缘规整，内部回声均匀，胰管未见扩张。,脾脏大小、形态正常，包膜光整，回声均匀。,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,膀胱充盈良好，壁光滑，延续性好，其内透声性良好，未见明显占位性病变。,前列腺稍大： 46mm× 40 mm× 30 mm，内回声尚均匀。未见明显异常回声。,...,None,None,None,None,None,详见纸质报告,详见纸质报告,None,None,None
000381f0069cbf7537e6aac8923034ae,None,左肾、右肾、膀胱、双侧输尿管未发现明显异常;子宫:绝经后子宫左附件、右附件未发现明显异常;肝...,肝脏形态大小正常，轮廓规整，肝内回声呈点状密集弥漫性增强，肝内管道结构尚清晰。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,None,脾脏大小、形态正常，包膜光整，回声均匀。,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,膀胱充盈良好，壁光滑，延续性好，其内透声性良好，未见明显占位性病变。,None,...,None,None,None,None,None,None,None,None,None,None
0003848ebd8d8163603760d53d975693,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，未见异常血流。;双侧甲状腺大小形态正常，包膜尚完...","左肾、右肾、膀胱、双侧输尿管未发现明显异常;双侧乳腺小叶增生;甲状腺双叶回声不均匀,建议甲功...",肝脏大小、形态正常，包膜光整，肝内血管走行较清晰，光点分布尚均匀，其内未见明显异常光团。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,胰腺大小、形态正常，边缘规整，内部回声均匀，胰管未见扩张。,脾脏大小、形态正常，包膜光整，内光点均匀。,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,膀胱充盈良好，壁光滑，延续性好，其内透声性良好，未见明显占位性病变。,None,...,None,None,None,None,None,None,None,None,None,None
000401cbf304d5a8bd862a81bacfa494,"双侧乳腺腺体层轻度增厚,内部结构稍紊乱,回声欠均匀，CDFI：未见异常血流。;双侧甲状腺形态...",肝:脂肪肝（轻度）胆、胰、脾、左肾、右肾未发现明显异常;双侧乳腺轻度增生;子宫:宫颈囊肿（多...,肝脏形态大小正常，轮廓规整，肝内回声呈点状密集弥漫性增强，肝内管道结构尚清晰。,胆囊大小正常，壁光滑，腔内暗区清晰，胆总管测值正常范围。,胰腺头、体、尾大小测值正常，内回声均匀。,脾脏大小测值正常，回声均匀，脾静脉测值正常。,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾形态大小正常，包膜光整，实质回声均匀。集合系统未见分离。,None,None,...,None,None,None,None,None,None,None,None,None,None
00044a586c249c05f0969e45ef03ab9d,"双侧乳腺腺体层增厚,内部结构紊乱,回声不均，右侧内可见一低回声区，范围约21×15mm，边界...","双侧乳腺小叶增生右乳腺低回声区，考虑结节,建议医院进一步检查;肝、胆、胰、脾未发现明显异常;...",肝脏大小、形态正常，包膜光整，肝内血管走行较清晰，光点分布尚均匀，其内未见明显异常光团。,胆囊大小、形态正常，囊壁光整，囊腔内透声好，胆总管无扩张。,胰腺大小、形态正常，边缘规整，内部回声均匀，胰管未见扩张。,脾脏大小、形态正常，包膜光整，内光点均匀。,左肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,右肾大小、形态正常，包膜光滑，肾实质回声均匀，集合系统未见明显分离。,膀胱充盈良好，壁光滑，延续性好，其内透声性良好，未见明显占位性病变。,None,...,None,None,None,None,None,None,None,None,None,None
